#Sequential Pattern Mining

In [2]:
import numpy as np
import pandas as pd
import re

In [8]:
# Load and inspect dataset
orders = pd.read_csv('/content/drive/MyDrive/DM/Data_Preparation/customer_supermarket_cleaned_orders.csv', sep='\t', index_col=0).reset_index()
orders['BasketDate'] = pd.to_datetime(orders.BasketDate, format='%Y/%m/%d %H:%M')
orders.head()

,BasketID,ProdID,BasketDate,CustomerCountry,Sale,CustomerID,ProdDescr,Qta,BasketDay,BasketMonth,BasketWeekOfYear,BasketYear,BasketWeekOfMonth,Tot
0,536365,21730,2010-12-01 08:26:00,United Kingdom,4.25,17850.0,GLASS STAR FROSTED T-LIGHT HOLDER,6,2,12,48,2010,1,25.50
1,536365,22752,2010-12-01 08:26:00,United Kingdom,7.65,17850.0,SET 7 BABUSHKA NESTING BOXES,2,2,12,48,2010,1,15.30
2,536365,71053,2010-12-01 08:26:00,United Kingdom,3.39,17850.0,WHITE METAL LANTERN,6,2,12,48,2010,1,20.34
3,536365,84029E,2010-12-01 08:26:00,United Kingdom,3.39,17850.0,RED WOOLLY HOTTIE WHITE HEART.,6,2,12,48,2010,1,20.34
4,536365,84029G,2010-12-01 08:26:00,United Kingdom,3.39,17850.0,KNITTED UNION FLAG HOT WATER BOTTLE,6,2,12,48,2010,1,20.34


In [9]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405715 entries, 0 to 405714
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   BasketID           405715 non-null  int64         
 1   ProdID             405715 non-null  object        
 2   BasketDate         405715 non-null  datetime64[ns]
 3   CustomerCountry    405715 non-null  object        
 4   Sale               405715 non-null  float64       
 5   CustomerID         344943 non-null  float64       
 6   ProdDescr          405715 non-null  object        
 7   Qta                405715 non-null  int64         
 8   BasketDay          405715 non-null  int64         
 9   BasketMonth        405715 non-null  int64         
 10  BasketWeekOfYear   405715 non-null  int64         
 11  BasketYear         405715 non-null  int64         
 12  BasketWeekOfMonth  405715 non-null  int64         
 13  Tot                405715 non-null  float64 

In [10]:
print("Number of distinct ProdIDs (in the original dataset): " + str(len(orders.ProdID.unique())))

Number of distinct ProdIDs (in the original dataset): 3727


##Prepare dataset

In [11]:
customer_purchases = []
# For all customers
for customerID in orders.CustomerID.unique():
  customer = orders[orders.CustomerID == customerID]
  customer_baskets = []

  # For all baskets
  for basketID in customer.BasketID.unique():
    basket = customer[customer.BasketID == basketID].ProdID.unique()
    new_basket = set()

    # For all products
    for item in basket:
      new_basket.add(re.sub('[A-Za-z]+', '', item))

    basket = list(new_basket)
    basket = sorted(basket)
    customer_baskets.append(basket)
  customer_purchases.append(customer_baskets)

In [13]:
print("Number of customers: " + str(len(customer_purchases)))
print("Number of distinct ProdIDs: " + str(len((set([item for customer in customer_purchases for basket in customer for item in basket])))))
print("Maximum length of a sequence: " + str(max([len(baskets_list) for baskets_list in customer_purchases])))
print("Maximum size of an element: " + str(max([len(basket) for baskets_list in customer_purchases for basket in baskets_list])))
print("Maximum size of a sequence: " + str(max([sum([len(basket) for basket in baskets_list]) for baskets_list in customer_purchases])))

Number of customers: 4173
Number of distinct ProdIDs: 3165
Maximum length of a sequence: 196
Maximum size of an element: 500
Maximum size of a sequence: 7278


##Apriori

In [14]:
import copy
import itertools
from collections import defaultdict
from operator import itemgetter

# #### Our dataset format
# An event is a list of strings.
# A sequence is a list of events.
# A dataset is a list of sequences.
# Thus, a dataset is a list of lists of lists of strings.
#
# E.g.
# dataset =  [
#    [["a"], ["a", "b", "c"], ["a", "c"], ["c"]],
#    [["a"], ["c"], ["b", "c"]],
#    [["a", "b"], ["d"], ["c"], ["b"], ["c"]],
#    [["a"], ["c"], ["b"], ["c"]]
# ]


# ### This is a simple recursive method that checks if subsequence is a subSequence of mainSequence

def isSubsequence(mainSequence, subSequence):
    subSequenceClone = list(subSequence)  # clone the sequence, because we will alter it
    return isSubsequenceRecursive(mainSequence, subSequenceClone)  # start recursion

"""
Function for the recursive call of isSubsequence, not intended for external calls
"""
def isSubsequenceRecursive(mainSequence, subSequenceClone, start=0):
    # Check if empty: End of recursion, all itemsets have been found
    if (not subSequenceClone):
        return True
    # retrieves element of the subsequence and removes is from subsequence
    firstElem = set(subSequenceClone.pop(0))
    # Search for the first itemset...
    for i in range(start, len(mainSequence)):
        if (set(mainSequence[i]).issuperset(firstElem)):
            # and recurse
            return isSubsequenceRecursive(mainSequence, subSequenceClone, i + 1)
    return False


# ### Size of sequences
"""
Computes the size of the sequence (sum of the size of the contained elements)
"""
def sequenceSize(sequence):
    return sum(len(i) for i in sequence)


# ### Support of a sequence
"""
Computes the support of a sequence in a dataset
"""
def countSupport(dataset, candidateSequence):
    return sum(1 for seq in dataset if isSubsequence(seq, candidateSequence))


# # AprioriAll
# ### 1 . Candidate Generation

# #### For a single pair:
"""
Generates one candidate of size k from two candidates of size (k-1) as used in the AprioriAll algorithm
"""
def generateCandidatesForPair(cand1, cand2):
    cand1Clone = copy.deepcopy(cand1)
    cand2Clone = copy.deepcopy(cand2)
    # drop the leftmost item from cand1:
    if (len(cand1[0]) == 1):
        cand1Clone.pop(0)
    else:
        cand1Clone[0] = cand1Clone[0][1:]
    # drop the rightmost item from cand2:
    if (len(cand2[-1]) == 1):
        cand2Clone.pop(-1)
    else:
        cand2Clone[-1] = cand2Clone[-1][:-1]

    # if the result is not the same, then we dont need to join
    if not cand1Clone == cand2Clone:
        return []
    else:
        newCandidate = copy.deepcopy(cand1)
        if (len(cand2[-1]) == 1):
            newCandidate.append(cand2[-1])
        else:
            newCandidate[-1].extend(cand2[-1][-1])
        return newCandidate


# #### For a set of candidates (of the last level):
"""
Generates the set of candidates of size k from the set of frequent sequences with size (k-1)
"""
def generateCandidates(lastLevelCandidates):
    k = sequenceSize(lastLevelCandidates[0]) + 1
    if (k == 2):
        flatShortCandidates = [item for sublist2 in lastLevelCandidates for sublist1 in sublist2 for item in sublist1]
        result = [[[a, b]] for a in flatShortCandidates for b in flatShortCandidates if b > a]
        result.extend([[[a], [b]] for a in flatShortCandidates for b in flatShortCandidates])
        return result
    else:
        candidates = []
        for i in range(0, len(lastLevelCandidates)):
            for j in range(0, len(lastLevelCandidates)):
                newCand = generateCandidatesForPair(lastLevelCandidates[i], lastLevelCandidates[j])
                if (not newCand == []):
                    candidates.append(newCand)
        candidates.sort()
        return candidates

# ### 2 . Candidate Checking
"""
Computes all direct subsequence for a given sequence.
A direct subsequence is any sequence that originates from deleting exactly one item from any element in the original sequence.
"""
def generateDirectSubsequences(sequence):
    result = []
    for i, itemset in enumerate(sequence):
        if (len(itemset) == 1):
            sequenceClone = copy.deepcopy(sequence)
            sequenceClone.pop(i)
            result.append(sequenceClone)
        else:
            for j in range(len(itemset)):
                sequenceClone = copy.deepcopy(sequence)
                sequenceClone[i].pop(j)
                result.append(sequenceClone)
    return result

"""
Prunes the set of candidates generated for size k given all frequent sequence of level (k-1), as done in AprioriAll
"""
def pruneCandidates(candidatesLastLevel, candidatesGenerated):
    return [cand for cand in candidatesGenerated if
            all(x in candidatesLastLevel for x in generateDirectSubsequences(cand))]

# ### Put it all together:
"""
The AprioriAll algorithm. Computes the frequent sequences in a seqeunce dataset for a given minSupport

Args:
    dataset: A list of sequences, for which the frequent (sub-)sequences are computed
    minSupport: The minimum support that makes a sequence frequent
    verbose: If true, additional information on the mining process is printed (i.e., candidates on each level)
Returns:
    A list of tuples (s, c), where s is a frequent sequence, and c is the count for that sequence
"""
def apriori(dataset, minSupport, verbose=False):
    global numberOfCountingOperations
    numberOfCountingOperations = 0
    Overall = []
    itemsInDataset = sorted(set([item for sublist1 in dataset for sublist2 in sublist1 for item in sublist2]))
    singleItemSequences = [[[item]] for item in itemsInDataset]
    singleItemCounts = [(i, countSupport(dataset, i)) for i in singleItemSequences if
                        countSupport(dataset, i) >= minSupport]
    Overall.append(singleItemCounts)
    if verbose:
      print("Result, lvl 1: " + str(Overall[0]))

    k = 1
    while (True):
        if not Overall[k - 1]:
            break
        # 1. Candidate generation
        candidatesLastLevel = [x[0] for x in Overall[k - 1]]
        candidatesGenerated = generateCandidates(candidatesLastLevel)
        # 2. Candidate pruning (using a "containsall" subsequences)
        candidatesPruned = [cand for cand in candidatesGenerated if
                            all(x in candidatesLastLevel for x in generateDirectSubsequences(cand))]
        # 3. Candidate checking
        candidatesCounts = [(i, countSupport(dataset, i)) for i in candidatesPruned]
        resultLvl = [(i, count) for (i, count) in candidatesCounts if (count >= minSupport)]
        if verbose:
            print("Candidates generated, lvl " + str(k + 1) + ": " + str(candidatesGenerated))
            print("Candidates pruned, lvl " + str(k + 1) + ": " + str(candidatesPruned))
            print("Result, lvl " + str(k + 1) + ": " + str(resultLvl))
        Overall.append(resultLvl)
        k = k + 1
    # "flatten" Overall
    Overall = Overall[:-1]
    Overall = [item for sublist in Overall for item in sublist]
    return Overall

In [15]:
one_item_patterns = apriori(customer_purchases, len(customer_purchases) // 10) # 10% (417)

In [16]:
one_item_patterns = sorted(one_item_patterns, key = lambda k: k[1], reverse=True)
print("Number of 1-item patterns supported by 10% of the customers: " + str(len(one_item_patterns)))
for i in range(len(one_item_patterns)):
  print(one_item_patterns[i])

Number of 1-item patterns supported by 10% of the customers: 30
([['85123']], 746)
([['85099']], 696)
([['22423']], 684)
([['47566']], 669)
([['22720']], 607)
([['21212']], 597)
([['84879']], 572)
([['22457']], 565)
([['22138']], 550)
([['22469']], 543)
([['23298']], 523)
([['22086']], 522)
([['22961']], 515)
([['20725']], 513)
([['22960']], 486)
([['21034']], 480)
([['22382']], 480)
([['20728']], 466)
([['22470']], 463)
([['23203']], 463)
([['22139']], 457)
([['22666']], 451)
([['23209']], 450)
([['20727']], 448)
([['23245']], 436)
([['22384']], 434)
([['21790']], 432)
([['85049']], 423)
([['22383']], 422)
([['22993']], 420)


In [18]:
two_items_patterns = apriori(customer_purchases, len(customer_purchases) // 20) # 5% (208)

In [ ]:
print(two_items_patterns)

In [ ]:
two_items_patterns = [(p, f) for (p, f) in two_items_patterns if sequenceSize(p) > 1] # Keep only the patterns with 2 items
two_items_patterns = sorted(two_items_patterns, key = lambda k: k[1], reverse=True)
print("Number of 2-items patterns supported by 5% of the customers: " + str(len(two_items_patterns)))
for i in range(len(two_items_patterns)):
  print(two_items_patterns[i])

In [ ]:
two_items_patterns = [(p, f) for (p, f) in two_items_patterns if sequenceSize(p) > 1] # Keep only the patterns with 2 items
two_items_patterns = sorted(two_items_patterns, key = lambda k: k[1], reverse=True)
print("Number of 2-items patterns supported by 5% of the customers: " + str(len(two_items_patterns)))
for i in range(len(two_items_patterns)):
  print(two_items_patterns[i])

Number of 2-items patterns supported by 5% of the customers: 39
([['85099'], ['85099']], 313)
([['85123'], ['85123']], 310)
([['22697', '22699']], 277)
([['20725'], ['20725']], 258)
([['22697', '22698']], 256)
([['22086', '22910']], 254)
([['22469', '22470']], 254)
([['23203', '85099']], 247)
([['22386', '85099']], 246)
([['20725', '22384']], 244)
([['23300', '23301']], 242)
([['20725', '20727']], 241)
([['20725', '20728']], 237)
([['20728', '22383']], 235)
([['20728', '22384']], 235)
([['21733', '85123']], 234)
([['47566'], ['47566']], 234)
([['20725', '22382']], 233)
([['20725', '22383']], 233)
([['20727', '22384']], 232)
([['82482', '82494']], 230)
([['22382', '22383']], 229)
([['22698', '22699']], 228)
([['22726', '22727']], 223)
([['20727'], ['20727']], 222)
([['20726', '22382']], 219)
([['20728', '22382']], 217)
([['22577', '22578']], 217)
([['84879'], ['84879']], 217)
([['21212', '21977']], 215)
([['85099'], ['23203']], 215)
([['20725', '85099']], 214)
([['20727', '22383']], 214

In [ ]:
# This takes few hours to run
three_items_patterns = apriori(customer_purchases, 150) # ~3%

In [ ]:
# These are the 3-items patterns found with the previous command
three_items_patterns = [
 ([['85099'], ['85099'], ['85099']], 178),
 ([['85123'], ['85123'], ['85123']], 166)]

In [ ]:
three_items_patterns = [(p, f) for (p, f) in three_items_patterns if sequenceSize(p) > 2]
three_items_patterns = sorted(three_items_patterns, key = lambda k: k[1], reverse=True)
print("Number of 3-items patterns supported by ~3% of the customers: " + str(len(three_items_patterns)))
for i in range(len(three_items_patterns)):
  print(three_items_patterns[i])

Number of 3-items patterns supported by ~3% of the customers: 2
([['85099'], ['85099'], ['85099']], 178)
([['85123'], ['85123'], ['85123']], 166)


##Analysis

In [ ]:
# Compute Lift for sequences
# defined as the support countof the sequence divided by the support count of a subsequence
def lift(data, sequence, subsequence):
  count_sequence = countSupport(data, sequence)
  count_subsequence = countSupport(data, subsequence)

  return count_sequence / count_subsequence

In [ ]:
print("Stats for ProdID 85099")
print("P(buy item) = " + str(lift(customer_purchases, [['85099']], [])))
print("P(buy item | item bought already) = " + str(lift(customer_purchases, [['85099'], ['85099']], [['85099']])))
print("P(buy item twice | item bought already) = " + str(lift(customer_purchases, [['85099'], ['85099'], ['85099']], [['85099']])))
print("P(buy item | item bought twice) = " + str(lift(customer_purchases, [['85099'], ['85099'], ['85099']], [['85099'], ['85099']])))

Stats for ProdID 85099
P(buy item) = 0.16682646212847554
P(buy item | item bought already) = 0.44971264367816094
P(buy item twice | item bought already) = 0.2557471264367816
P(buy item | item bought twice) = 0.5686900958466453


In [ ]:
print("Stats for ProdID 85123")
print("P(buy item) = " + str(lift(customer_purchases, [['85123']], [])))
print("P(buy item | item bought already) = " + str(lift(customer_purchases, [['85123'], ['85123']], [['85123']])))
print("P(buy item twice | item bought already) = " + str(lift(customer_purchases, [['85123'], ['85123'], ['85123']], [['85123']])))
print("P(buy item | item bought twice) = " + str(lift(customer_purchases, [['85123'], ['85123'], ['85123']], [['85123'], ['85123']])))

Stats for ProdID 85123
P(buy item) = 0.1788111217641419
P(buy item | item bought already) = 0.4155495978552279
P(buy item twice | item bought already) = 0.2225201072386059
P(buy item | item bought twice) = 0.535483870967742


In [ ]:
# Divide 2-items patterns between single element patterns and 2 elements patterns
two_items_single = [p for p in two_items_patterns if len(p[0]) == 1]
two_items_double = [p for p in two_items_patterns if len(p[0]) == 2]

print("Number of 2-items patterns made of 1 element: " + str(len(two_items_single)))
print("Number of 2-items patterns made of 2 elements: " + str(len(two_items_double)))

Number of 2-items patterns made of 1 element: 31
Number of 2-items patterns made of 2 elements: 8


In [ ]:
# Inspect patterns made of 2 elements
two_items_double

[([['85099'], ['85099']], 313),
 ([['85123'], ['85123']], 310),
 ([['20725'], ['20725']], 258),
 ([['47566'], ['47566']], 234),
 ([['20727'], ['20727']], 222),
 ([['84879'], ['84879']], 217),
 ([['85099'], ['23203']], 215),
 ([['22383'], ['22383']], 212)]

In [ ]:
# Compute Lift for the patterns with the same item in the two elements
for id in ['85099', '85123', '20725', '47566', '20727', '84879', '22383']:
  count_single =  countSupport(customer_purchases, [[id]])
  count_double = countSupport(customer_purchases, [[id], [id]])
  print("ProdID " + id)
  print("\t\t                  P(id) = " + str(count_single / len(customer_purchases)))
  print("\t\t P(id | already bought) = " + str(count_double / count_single))

ProdID 85099
		                  P(id) = 0.16682646212847554
		 P(id | already bought) = 0.44971264367816094
ProdID 85123
		                  P(id) = 0.1788111217641419
		 P(id | already bought) = 0.4155495978552279
ProdID 20725
		                  P(id) = 0.12296260786193672
		 P(id | already bought) = 0.5029239766081871
ProdID 47566
		                  P(id) = 0.16035474592521573
		 P(id | already bought) = 0.34977578475336324
ProdID 20727
		                  P(id) = 0.10738255033557047
		 P(id | already bought) = 0.4955357142857143
ProdID 84879
		                  P(id) = 0.137104506232023
		 P(id | already bought) = 0.3793706293706294
ProdID 22383
		                  P(id) = 0.10115052732502397
		 P(id | already bought) = 0.5023696682464455


In [ ]:
# Inspect the product(s) corresponding to those ProdID
for id in ['85099', '85123', '20725', '47566', '20727', '84879', '22383']:
  print(orders[orders.ProdID.str.contains(id)].groupby(["ProdID","ProdDescr"]).Sale.min().reset_index())
  print("")

   ProdID                       ProdDescr  Sale
0  85099B         JUMBO BAG RED RETROSPOT  1.65
1  85099C  JUMBO  BAG BAROQUE BLACK WHITE  1.65
2  85099F            JUMBO BAG STRAWBERRY  1.65

   ProdID                           ProdDescr  Sale
0  85123A  CREAM HANGING HEART T-LIGHT HOLDER  2.95
1  85123A  WHITE HANGING HEART T-LIGHT HOLDER  2.55

  ProdID                ProdDescr  Sale
0  20725  LUNCH BAG RED RETROSPOT  1.45

   ProdID               ProdDescr  Sale
0   47566           PARTY BUNTING  4.15
1  47566B  TEA TIME PARTY BUNTING  4.65

  ProdID                ProdDescr  Sale
0  20727  LUNCH BAG  BLACK SKULL.  1.45

  ProdID                      ProdDescr  Sale
0  84879  ASSORTED COLOUR BIRD ORNAMENT  1.69

  ProdID                ProdDescr  Sale
0  22383  LUNCH BAG SUKI  DESIGN   1.65
1  22383   LUNCH BAG SUKI DESIGN   1.65



In [ ]:
# Check the products in the only pattern with 2 distinct items in the two elements (ID 85099)
orders[orders.ProdID.str.contains("85099")].groupby(["ProdID","ProdDescr"]).Sale.min().reset_index()

,ProdID,ProdDescr,Sale
0,85099B,JUMBO BAG RED RETROSPOT,1.65
1,85099C,JUMBO BAG BAROQUE BLACK WHITE,1.65
2,85099F,JUMBO BAG STRAWBERRY,1.65


In [ ]:
# Check the products in the only pattern with 2 distinct items in the two elements (ID 23203)
orders[orders.ProdID.str.contains("23203")].groupby(["ProdID","ProdDescr"]).Sale.min().reset_index()

,ProdID,ProdDescr,Sale
0,23203,JUMBO BAG DOILEY PATTERNS,1.95
1,23203,JUMBO BAG VINTAGE DOILEY,2.08
2,23203,JUMBO BAG VINTAGE DOILY,1.79


In [ ]:
# Inspect patterns made of 2 elements
two_items_single

[([['22697', '22699']], 277),
 ([['22697', '22698']], 256),
 ([['22086', '22910']], 254),
 ([['22469', '22470']], 254),
 ([['23203', '85099']], 247),
 ([['22386', '85099']], 246),
 ([['20725', '22384']], 244),
 ([['23300', '23301']], 242),
 ([['20725', '20727']], 241),
 ([['20725', '20728']], 237),
 ([['20728', '22383']], 235),
 ([['20728', '22384']], 235),
 ([['21733', '85123']], 234),
 ([['20725', '22382']], 233),
 ([['20725', '22383']], 233),
 ([['20727', '22384']], 232),
 ([['82482', '82494']], 230),
 ([['22382', '22383']], 229),
 ([['22698', '22699']], 228),
 ([['22726', '22727']], 223),
 ([['20726', '22382']], 219),
 ([['20728', '22382']], 217),
 ([['22577', '22578']], 217),
 ([['21212', '21977']], 215),
 ([['20725', '85099']], 214),
 ([['20727', '22383']], 214),
 ([['22138', '22617']], 214),
 ([['20727', '22382']], 212),
 ([['21212', '84991']], 212),
 ([['23298', '47566']], 212),
 ([['22411', '85099']], 210)]

In [ ]:
# Get number of distinct items in the patterns
items = set()
for (p, f) in two_items_single:
  items.add(p[0][0])
  items.add(p[0][1])

items = list(items)
print("Number of distinct items: " + str(len(items)))

Number of distinct items: 35


In [ ]:
for pattern, _ in two_items_single:
  descr_1 = orders[orders.ProdID.str.contains(pattern[0][0])].groupby(["ProdID"]).ProdDescr.min()
  descr_2 = orders[orders.ProdID.str.contains(pattern[0][1])].groupby(["ProdID"]).ProdDescr.min()

  print(descr_1)
  print(descr_2)
  print("")

ProdID
22697    GREEN REGENCY TEACUP AND SAUCER
Name: ProdDescr, dtype: object
ProdID
22699    ROSES REGENCY TEACUP AND SAUCER 
Name: ProdDescr, dtype: object

ProdID
22697    GREEN REGENCY TEACUP AND SAUCER
Name: ProdDescr, dtype: object
ProdID
22698    PINK REGENCY TEACUP AND SAUCER
Name: ProdDescr, dtype: object

ProdID
22086    PAPER CHAIN KIT 50'S CHRISTMAS 
Name: ProdDescr, dtype: object
ProdID
22910    PAPER CHAIN KIT VINTAGE CHRISTMAS
Name: ProdDescr, dtype: object

ProdID
22469    HEART OF WICKER SMALL
Name: ProdDescr, dtype: object
ProdID
22470    HEART OF WICKER LARGE
Name: ProdDescr, dtype: object

ProdID
23203    JUMBO BAG DOILEY PATTERNS
Name: ProdDescr, dtype: object
ProdID
85099B           JUMBO BAG RED RETROSPOT
85099C    JUMBO  BAG BAROQUE BLACK WHITE
85099F              JUMBO BAG STRAWBERRY
Name: ProdDescr, dtype: object

ProdID
22386    JUMBO BAG PINK POLKADOT
Name: ProdDescr, dtype: object
ProdID
85099B           JUMBO BAG RED RETROSPOT
85099C    JUMBO  BAG BAROQUE

#REMEMBER TO REMOVE THIS


In [ ]:
pt_2 = [
 ([['20725', '20726']], 205), ([['20725', '20727']], 241), ([['20725', '20728']], 237), ([['20725', '22382']], 233), ([['20725', '22383']], 233),
 ([['20725', '22384']], 244), ([['20725', '22662']], 161), ([['20725', '23206']], 183), ([['20725', '23207']], 152), ([['20725', '23209']], 197),
 ([['20725', '85099']], 214), ([['20726', '20727']], 175), ([['20726', '20728']], 192), ([['20726', '22382']], 219), ([['20726', '22383']], 173),
 ([['20726', '22384']], 166), ([['20726', '23206']], 154), ([['20727', '20728']], 199), ([['20727', '22382']], 212), ([['20727', '22383']], 214),
 ([['20727', '22384']], 232), ([['20727', '23206']], 163), ([['20727', '23207']], 153), ([['20727', '23209']], 176), ([['20727', '85099']], 161),
 ([['20728', '22382']], 217), ([['20728', '22383']], 235), ([['20728', '22384']], 235), ([['20728', '23206']], 166), ([['20728', '23207']], 155),
 ([['20728', '23209']], 159), ([['20728', '85099']], 168), ([['21136', '84879']], 150), ([['21166', '21175']], 153), ([['21212', '21975']], 156),
 ([['21212', '21977']], 215), ([['21212', '84991']], 212), ([['21733', '85123']], 234), ([['21754', '21755']], 168), ([['21790', '21791']], 174),
 ([['21914', '21915']], 165), ([['21928', '85099']], 176), ([['21929', '85099']], 199), ([['21931', '85099']], 203), ([['21977', '84991']], 172),
 ([['22077', '85049']], 174), ([['22086', '22910']], 254), ([['22112', '22835']], 160), ([['22138', '22139']], 158), ([['22138', '22617']], 214),
 ([['22142', '22144']], 159), ([['22382', '22383']], 229), ([['22382', '22384']], 196), ([['22382', '22662']], 189), ([['22382', '23206']], 167),
 ([['22382', '23207']], 153), ([['22382', '23209']], 173), ([['22383', '22384']], 194), ([['22383', '23206']], 177), ([['22383', '23207']], 152),
 ([['22383', '23209']], 169), ([['22383', '85099']], 158), ([['22384', '23206']], 155), ([['22384', '23209']], 167), ([['22384', '85099']], 172),
 ([['22386', '85099']], 246), ([['22411', '85099']], 210), ([['22423', '22697']], 151), ([['22423', '22699']], 166), ([['22457', '85123']], 156),
 ([['22469', '22470']], 254), ([['22469', '85123']], 160), ([['22470', '85123']], 173), ([['22551', '22554']], 179), ([['22554', '22556']], 155),
 ([['22577', '22578']], 217), ([['22578', '22579']], 171), ([['22624', '22625']], 167), ([['22629', '22630']], 204), ([['22666', '22720']], 176),
 ([['22697', '22698']], 256), ([['22697', '22699']], 277), ([['22698', '22699']], 228), ([['22720', '22722']], 201), ([['22720', '23243']], 154),
 ([['22726', '22727']], 223), ([['22726', '22728']], 168), ([['22727', '22728']], 188), ([['22727', '22730']], 164), ([['22745', '22748']], 175),
 ([['22746', '22748']], 153), ([['22749', '22750']], 150), ([['22804', '85123']], 176), ([['22865', '22866']], 179), ([['22865', '22867']], 178),
 ([['22866', '22867']], 157), ([['22960', '22961']], 180), ([['23199', '23200']], 153), ([['23199', '23203']], 155), ([['23199', '23206']], 152),
 ([['23199', '85099']], 177), ([['23201', '23203']], 155), ([['23201', '85099']], 190), ([['23202', '23203']], 190), ([['23202', '85099']], 182),
 ([['23203', '23209']], 202), ([['23203', '85099']], 247), ([['23206', '23207']], 182), ([['23206', '23208']], 165), ([['23206', '23209']], 194),
 ([['23207', '23209']], 158), ([['23208', '23209']], 169), ([['23209', '85099']], 157), ([['23254', '23256']], 159), ([['23293', '23294']], 151),
 ([['23293', '23295']], 171), ([['23293', '23296']], 158), ([['23298', '47566']], 212), ([['23300', '23301']], 242), ([['23321', '23322']], 201),
 ([['23343', '23344']], 150), ([['47566', '85123']], 152), ([['82482', '82494']], 230), ([['84378', '84380']], 176), ([['20725'], ['20725']], 258),
 ([['20725'], ['20726']], 166), ([['20725'], ['20727']], 201), ([['20725'], ['20728']], 181), ([['20725'], ['22382']], 200), ([['20725'], ['22383']], 194),
 ([['20725'], ['22384']], 185), ([['20725'], ['23203']], 154), ([['20725'], ['23206']], 182), ([['20725'], ['23207']], 162), ([['20725'], ['23209']], 198),
 ([['20725'], ['85099']], 192), ([['20726'], ['20725']], 165), ([['20726'], ['20726']], 181), ([['20726'], ['20727']], 158), ([['20726'], ['22382']], 160),
 ([['20727'], ['20725']], 193), ([['20727'], ['20727']], 222), ([['20727'], ['20728']], 165), ([['20727'], ['22382']], 166), ([['20727'], ['22383']], 174),
 ([['20727'], ['22384']], 165), ([['20727'], ['23209']], 162), ([['20727'], ['85099']], 166), ([['20728'], ['20725']], 199), ([['20728'], ['20727']], 181),
 ([['20728'], ['20728']], 193), ([['20728'], ['22382']], 171), ([['20728'], ['22383']], 178), ([['20728'], ['22384']], 177), ([['20728'], ['23206']], 158),
 ([['20728'], ['23207']], 164), ([['20728'], ['23209']], 173), ([['20728'], ['85099']], 165), ([['21034'], ['21034']], 178), ([['21212'], ['21212']], 165),
 ([['22197'], ['22197']], 155), ([['22382'], ['20725']], 188), ([['22382'], ['20726']], 160), ([['22382'], ['20727']], 169), ([['22382'], ['20728']], 158),
 ([['22382'], ['22382']], 192), ([['22382'], ['22383']], 176), ([['22382'], ['22384']], 154), ([['22382'], ['23209']], 165), ([['22382'], ['85099']], 150),
 ([['22383'], ['20725']], 182), ([['22383'], ['20727']], 182), ([['22383'], ['20728']], 168), ([['22383'], ['22382']], 177), ([['22383'], ['22383']], 212),
 ([['22383'], ['22384']], 156), ([['22383'], ['23206']], 156), ([['22383'], ['23207']], 150), ([['22383'], ['23209']], 156), ([['22383'], ['85099']], 152),
 ([['22384'], ['20725']], 189), ([['22384'], ['20727']], 173), ([['22384'], ['20728']], 165), ([['22384'], ['22382']], 167), ([['22384'], ['22383']], 158),
 ([['22384'], ['22384']], 186), ([['22384'], ['23206']], 152), ([['22384'], ['23207']], 155), ([['22384'], ['23209']], 162), ([['22384'], ['85099']], 174),
 ([['22386'], ['85099']], 173), ([['22411'], ['85099']], 152), ([['22423'], ['22423']], 184), ([['22457'], ['22457']], 168), ([['22469'], ['22469']], 178),
 ([['22469'], ['85123']], 151), ([['22470'], ['22470']], 151), ([['22720'], ['22720']], 199), ([['22727'], ['22727']], 163), ([['22993'], ['22993']], 164),
 ([['23201'], ['85099']], 152), ([['23203'], ['23203']], 180), ([['23203'], ['85099']], 179), ([['23206'], ['20725']], 151), ([['23206'], ['23206']], 175),
 ([['23206'], ['23209']], 155), ([['23207'], ['23209']], 154), ([['23209'], ['20725']], 155), ([['23209'], ['23203']], 152), ([['23209'], ['23209']], 197),
 ([['23298'], ['23298']], 154), ([['47566'], ['23298']], 179), ([['47566'], ['47566']], 234), ([['82482'], ['82482']], 162), ([['82482'], ['82494']], 158),
 ([['82494'], ['82494']], 152), ([['84879'], ['84879']], 217), ([['85099'], ['20725']], 197), ([['85099'], ['20727']], 174), ([['85099'], ['22382']], 153),
 ([['85099'], ['22383']], 151), ([['85099'], ['22384']], 163), ([['85099'], ['22386']], 169), ([['85099'], ['23199']], 175), ([['85099'], ['23201']], 168),
 ([['85099'], ['23202']], 175), ([['85099'], ['23203']], 215), ([['85099'], ['23206']], 154), ([['85099'], ['23209']], 170), ([['85099'], ['23344']], 167),
 ([['85099'], ['85099']], 313), ([['85123'], ['85099']], 157), ([['85123'], ['85123']], 310)]

two_items_patterns = [(p, f) for (p, f) in pt_2 if f > len(customer_purchases) // 20] # 5% (208)